# Fractional Flow


In [1]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from ipywidgets import BoundedFloatText, interactive, interact,FloatSlider
%matplotlib inline
    
def fracflow(Swr,Sor,krwo,kroo,visco,u,alpha,rhoo,k,no,nw):
    
    viscw = 1 # cp
    rhow = 62.4 # lb/ft3
    
    Swr2 = 1-Sor

    Sw = np.linspace(Swr,Swr2,100)
    Sw1 = Sw + 0.0001

    krw = np.empty(100)
    kro = np.empty(100) 
    krw1 = np.empty(100)
    kro1 = np.empty(100)
    
    for i,j in enumerate(Sw):
        
        S = (j-Swr)/(1-Swr-Sor)
        krw[i] = krwo*(S**nw)
        kro[i] = kroo*((1-S)**no)
         
    fw = np.empty(100)
    dfw = np.empty(100)
    
    for i,j in enumerate(Sw):
        
        with np.errstate(divide='ignore'):
        
            bottom = (1+(kro[i]*viscw)/(krw[i]*visco))

            fw[i]=(1+(6.33E-3/144)*((k*kro[i])/(u*visco))*((rhoo-rhow)*m.sin(np.deg2rad(alpha))))/bottom
                   
            
            if i == 0 :
                
                dfw[i] = 0
                
            else:
                
                dfw[i]=(fw[i]-fw[i-1])/(Sw[i]-Sw[i-1])
                
    
    f = plt.figure(figsize=(15,5))
    ax = f.add_subplot(121)
    ax2 = f.add_subplot(122)
    ax.plot(Sw,fw,'b-')
    ax.set_title('Fractional Flow of Water')
    ax.set_xlabel('$S_W$')
    ax.set_ylabel('$f_W$')
    ax.grid()
    ax2.plot(Sw,dfw,'g-')
    ax2.set_title('Derivative of Fractional Flow')
    ax2.set_xlabel('$S_W$')
    ax2.set_ylabel('$df_W/dS_W$')
    ax2.grid() 
    
InputSwr = BoundedFloatText(value=0.25,min=0,max=1,step=0.01,description='Swr',disabled=False)
InputSor = BoundedFloatText(value=0.3,min=0,max=1,step=0.01,description='Sor',disabled=False)
Inputkrwo = BoundedFloatText(value=0.2,min=0,max=1,step=0.01,description='krw0',disabled=False)
Inputkroo = BoundedFloatText(value=1,min=0,max=1,step=0.01,description='kro0',disabled=False)
Inputvisco = BoundedFloatText(value=5,min=0,max=150,step=0.1,description='Visc [cp]',disabled=False)
Inputu = BoundedFloatText(value=0.01,min=0,max=100,step=0.01,description='u [ft/day]',disabled=False)
Inputalpha = BoundedFloatText(value=0,min=-180,max=180,step=0.01,description='α [deg]',disabled=False)
Inputrhoo = BoundedFloatText(value=40.5,min=0,max=62.4,step=0.01,description='ρ_o [lb/ft^3]',disabled=False)
Inputk = BoundedFloatText(value=416,min=0,max=1000,step=0.01,description='Perm [mD]',disabled=False)

interact (fracflow,Swr=InputSwr, 
          Sor=InputSor,
          krwo=Inputkrwo,
          kroo=Inputkroo,
          visco=Inputvisco,
          u=Inputu,
          alpha=Inputalpha,
          rhoo=Inputrhoo,
          k=Inputk,
                 
          no=FloatSlider(value=2,min=1,max=6.0,step=0.01,disabled=False,continuous_update=False,
                             orientation='horizontal',
                             layout = {'width':'40%'},
                             readout=True,
                             readout_format='.2f'),
          nw=FloatSlider(value=2,min=1,max=6.0,step=0.01,disabled=False,continuous_update=False,
                             orientation='horizontal',
                             layout = {'width':'40%'},
                             readout=True,
                             readout_format='.2f'));
    

interactive(children=(BoundedFloatText(value=0.25, description='Swr', max=1.0, step=0.01), BoundedFloatText(va…